Một trong những yếu tố đằng sau sự thành công của học sâu là sự sẵn có của nhiều lớp khác nhau, có thể được kết hợp theo nhiều cách sáng tạo để thiết kế các kiến ​​trúc phù hợp với nhiều nhiệm vụ khác nhau. Ví dụ, các nhà nghiên cứu đã phát minh ra các lớp chuyên dụng để xử lý hình ảnh, văn bản, lặp qua dữ liệu tuần tự và thực hiện lập trình động. Sớm hay muộn, bạn sẽ cần một lớp chưa tồn tại trong khung học sâu. Trong những trường hợp này, bạn phải xây dựng một lớp tùy chỉnh. Trong phần này, chúng tôi sẽ hướng dẫn bạn cách thực hiện.

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
# custom layer không có tham số, tính giá trị sau khi trừ đi mean()

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [3]:
# tích hợp để xây dựng một mô hình phức tạp hơn
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()  # gần bằng 0

tensor(3.7253e-09, grad_fn=<MeanBackward0>)

In [5]:
# định nghĩa các lớp có param ó thể điều chỉnh thông qua quá trình huhaasn luyện
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-1.0216, -0.2803, -1.9763],
        [-1.0568,  0.4503, -0.1269],
        [-0.3968, -1.5294, -1.4176],
        [ 0.1990,  1.3232, -1.5355],
        [ 0.5873,  0.1324,  0.5814]], requires_grad=True)

In [6]:
# lan truyền tiến
linear(torch.rand(2, 5))

tensor([[0.0000, 0.8620, 0.0000],
        [0.0000, 0.0784, 0.0000]])

In [7]:
# tích hợp vào mô hình phức tạp hơn
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[ 4.0980],
        [13.7104]])